In [1]:
import numpy as np
import pandas as pd

from libreco.data import DatasetPure
from libreco.algorithms import BPR
from libreco.evaluation import evaluate

from sklearn.model_selection import train_test_split

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
books_df = pd.read_csv('../data/Books.csv', delimiter=';', low_memory=False)
ratings_df = pd.read_csv('../data/Ratings.csv', delimiter=';')
users_df = pd.read_csv('../data/Users.csv', delimiter=';')

/var/folders/s0/4m_t_82936ggv17n0g0t7_dr0000gn/T/ipykernel_18292/4047143737.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users_df = pd.read_csv('../data/Users.csv', delimiter=';')


In [3]:
ratings_df.columns = ['user_id', 'item_id', 'label']

In [4]:
ratings_df = ratings_df[ratings_df['label'] != 0]

In [5]:
book_str_to_int = {book_id: i for i, book_id in enumerate(ratings_df['item_id'].unique())}
user_str_to_int = {user_id: i for i, user_id in enumerate(ratings_df['user_id'].unique())}

ratings_df['item'] = ratings_df['item_id'].map(book_str_to_int).to_numpy()
ratings_df['user'] = ratings_df['user_id'].map(user_str_to_int).to_numpy()



In [6]:
train_df, eval_df = train_test_split(ratings_df, test_size=0.2, random_state=0)

In [10]:
eval_user_mask = eval_df['user'].isin(train_df['user'].unique())
eval_item_mask = eval_df['item'].isin(train_df['item'].unique())

eval_df = eval_df[eval_user_mask & eval_item_mask]

# 'user', 'item' must be the first two columns of the dataframe
train_df = train_df[['user', 'item', 'label']]
eval_df = eval_df[['user', 'item', 'label']]

In [11]:
eval_df.shape

(51413, 3)

In [12]:

# Step 2: Prepare the dataset for LibRecommender
train_data, data_info = DatasetPure.build_trainset(train_df)
eval_data = DatasetPure.build_evalset(eval_df)



In [14]:

# Step 3: Initialize the BPR model
model = BPR(task="ranking",  # This specifies that we're performing a ranking task
            data_info=data_info,
            embed_size=16,  # Size of the embedding vectors
            n_epochs=10,  # Number of training epochs
            lr=0.01,  # Learning rate
            reg=None,  # Regularization parameter, can be tuned
            batch_size=256,  # Batch size for training
            num_neg=1,  # Number of negative samples per positive sample
            use_tf=True,  # Whether to use TensorFlow backend
            seed=42)

In [15]:
# Step 4: Train the model
model.fit(
    train_data, 
    neg_sampling=True,
    verbose=2, 
    shuffle=True, 
    eval_data=eval_data, 
    metrics=["ndcg", "precision"]
)


2024-08-09 22:58:05.144491: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


Training start time: 2024-08-09 22:58:05


train: 100%|██████████| 1356/1356 [00:11<00:00, 122.15it/s]


Epoch 1 elapsed: 11.118s
	 train_loss: 0.6237


eval_listwise: 100%|██████████| 16331/16331 [01:16<00:00, 213.15it/s]


...

	 eval ndcg@10: 0.0086
	 eval precision@10: 0.0018


train: 100%|██████████| 1356/1356 [00:10<00:00, 129.23it/s]


Epoch 10 elapsed: 10.500s
	 train_loss: 0.0419


eval_listwise: 100%|██████████| 16331/16331 [01:29<00:00, 182.61it/s]


	 eval ndcg@10: 0.0082
	 eval precision@10: 0.0018


In [16]:
# Step 5: Evaluate the model
eval_result = evaluate(model, eval_data, neg_sampling=True, metrics=["ndcg", "precision", "recall"])
print(f"Evaluation Results:\n{eval_result}")

eval_listwise: 100%|██████████| 16331/16331 [00:57<00:00, 282.05it/s]


Evaluation Results:
{'ndcg': 0.008218662924561774, 'precision': 0.001818627150817464, 'recall': 0.008496358810695952}


In [17]:
user_id = train_df.iloc[:10]['user'].values
item_id = train_df.iloc[:10]['item'].values
prediction = model.predict(user_id, item_id)
print(f"Prediction for user {user_id} and item {item_id}: {prediction}")

Prediction for user [44828 13679 67574  5478 50595 44797  9787 67068    64 76536] and item [129010  52858 127059  27618 139955   4218  42336  43591    202   3399]: [0.998423   0.9999753  0.99999416 0.99969935 0.99830866 0.9891834
 0.99997854 0.9968299  0.9996921  0.99963665]
